In [7]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from PIL import Image
import pandas as pd
import os
import timm

# 모델 아키텍처 설정
model1 = timm.create_model("tf_efficientnet_b0", pretrained=True)
model2 = timm.create_model("tf_efficientnet_b0", pretrained=True)
model3 = timm.create_model("tf_efficientnet_b0", pretrained=True)
model4 = timm.create_model("tf_efficientnet_b0", pretrained=True)

num_classes = 2
in_features = model1.classifier.in_features
model1.classifier = nn.Linear(in_features, num_classes)
model2.classifier = nn.Linear(in_features, num_classes)
model3.classifier = nn.Linear(in_features, num_classes)
model4.classifier = nn.Linear(in_features, num_classes)

# 저장된 상태 사전 로드
state_dict1 = torch.load("HF.pt", map_location=torch.device('cpu'))
state_dict2 = torch.load("HN.pt", map_location=torch.device('cpu'))
state_dict3 = torch.load("LF.pt", map_location=torch.device('cpu'))
state_dict4 = torch.load("LN.pt", map_location=torch.device('cpu'))

# 로드한 상태 사전에서 불필요한 키 제거
state_dict1.pop("classifier.weight", None)
state_dict1.pop("classifier.bias", None)
state_dict2.pop("classifier.weight", None)
state_dict2.pop("classifier.bias", None)
state_dict3.pop("classifier.weight", None)
state_dict3.pop("classifier.bias", None)
state_dict4.pop("classifier.weight", None)
state_dict4.pop("classifier.bias", None)

# 수정된 모델 아키텍처로 상태 사전 로드
model1.load_state_dict(state_dict1, strict=False)
model2.load_state_dict(state_dict2, strict=False)
model3.load_state_dict(state_dict3, strict=False)
model4.load_state_dict(state_dict4, strict=False)

# 모델을 평가 모드로 설정
model1.eval()
model2.eval()
model3.eval()
model4.eval()

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# CSV 파일 로드
csv_path = "your_prediction.csv"  # CSV 파일 경로
df = pd.read_csv(csv_path)

# 예측 수행 및 결과 저장
predictions = []

# 폴더 경로
base_folder = "C:/Users/USER/Desktop/2017310463"  # 자신의 데스크탑 경로로 변경해주세요

# 폴더 순회
for video_folder in os.listdir(base_folder):
    video_folder_path = os.path.join(base_folder, video_folder)
    if not os.path.isdir(video_folder_path):
        continue
    
    # 각 비디오 폴더 순회
    for face_file in os.listdir(video_folder_path):
        if not face_file.startswith("face_") or not face_file.endswith(".png"):
            continue
        
        # 이미지 파일 경로
        image_path = os.path.join(video_folder_path, face_file)
        
        # 이미지 로드 및 전처리
        image = Image.open(image_path)
        image = transform(image)
        image = image.unsqueeze(0)  # 배치 차원 추가
        
        # 예측 수행
        with torch.no_grad():
            outputs1 = model1(image)
            outputs2 = model2(image)
            outputs3 = model3(image)
            outputs4 = model4(image)
        
        probabilities1 = nn.functional.softmax(outputs1, dim=1)[0]
        probabilities2 = nn.functional.softmax(outputs2, dim=1)[0]
        probabilities3 = nn.functional.softmax(outputs3, dim=1)[0]
        probabilities4 = nn.functional.softmax(outputs4, dim=1)[0]
        
        predicted_label1 = torch.argmax(probabilities1)
        predicted_label2 = torch.argmax(probabilities2)
        predicted_label3 = torch.argmax(probabilities3)
        predicted_label4 = torch.argmax(probabilities4)
        
        # 각 모델의 예측 결과를 리스트에 추가
        predictions.append([predicted_label1.item(), predicted_label2.item(), predicted_label3.item(), predicted_label4.item()])

# 예측 결과 업데이트
df["predicted_labels"] = predictions

# 모델별 예측 정확도 계산
model1_accuracy = sum(df["label"] == df["predicted_labels"].apply(lambda x: x[0])) / len(df)
model2_accuracy = sum(df["label"] == df["predicted_labels"].apply(lambda x: x[1])) / len(df)
model3_accuracy = sum(df["label"] == df["predicted_labels"].apply(lambda x: x[2])) / len(df)
model4_accuracy = sum(df["label"] == df["predicted_labels"].apply(lambda x: x[3])) / len(df)

# 결과 출력
print("Model 1 Accuracy:", model1_accuracy)
print("Model 2 Accuracy:", model2_accuracy)
print("Model 3 Accuracy:", model3_accuracy)
print("Model 4 Accuracy:", model4_accuracy)

# 결과 저장
output_csv_path = "path_to_output_csv_file.csv"  # 결과를 저장할 CSV 파일 경로
df.to_csv(output_csv_path, index=False)


Model 1 Accuracy: 0.4666666666666667
Model 2 Accuracy: 0.36
Model 3 Accuracy: 0.38666666666666666
Model 4 Accuracy: 0.4
